This is an attempt to recreate the parameter estimation [example](https://sites.engineering.ucsb.edu/~jbraw/chemreacfun/fig-html/appendix/fig-A-10.html) from James Rawlings book on [Reactor Design](https://sites.engineering.ucsb.edu/~jbraw/chemreacfun/) using Pyomo.

In [1]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pyomo.environ as pyo
from pyomo.contrib.interior_point.inverse_reduced_hessian import inv_reduced_hessian_barrier

In [5]:
k = 0.4
c0 = 1.0
c1 = c0 * (1.0 - k)
c2 = c1 * (1.0 - k)
print(c0, c1, c2)

1.0 0.6 0.36


In [6]:
cexp = [0.7, 0.2]

In [51]:
#
# Define the model 
#
def toy_model(cexp):
       
    c0 = 1.0
      
    m = pyo.ConcreteModel()
    
    m.k1 = pyo.Var(initialize = 0.5, bounds = (1e-4, 2.0))
    
    m.c0_init = pyo.Var(initialize = c0, bounds = (0.0, c0))
    m.c0 = pyo.Var(initialize = c0, bounds = (0.0, c0))
    m.c1 = pyo.Var(initialize = c0, bounds = (0.0, c0))
    m.c2 = pyo.Var(initialize = c0, bounds = (0.0, c0))
    
    m.c0_cons = pyo.Constraint(expr = m.c0 == m.c0_init)
    #m.c0.fix(c0)
    m.c1_cons = pyo.Constraint(expr = m.c1 == c0 * (1.0 - m.k1))
    m.c2_cons = pyo.Constraint(expr = m.c2 == m.c1 * (1.0 - m.k1))
    

    def obj_rule(m):
        return (m.c1 - cexp[0]) ** 2 + (m.c2 - cexp[1]) ** 2  
    m.obj = pyo.Objective(rule=obj_rule)
    
    return m

In [52]:
m = toy_model(cexp)
solver = pyo.SolverFactory('ipopt')
solver.solve(m, tee = True)

Ipopt 3.14.5: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.5, running with linear solver ma27.

Number of nonzeros in equality constraint Jacobian...:        7
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        3

Total number of variables............................:        5
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        5
                     variables with only upper bounds:        0
Total number of equality constraints.................:        3
Total number of inequ

{'Problem': [{'Lower bound': -inf, 'Upper bound': inf, 'Number of objectives': 1, 'Number of constraints': 3, 'Number of variables': 5, 'Sense': 'unknown'}], 'Solver': [{'Status': 'ok', 'Message': 'Ipopt 3.14.5\\x3a Optimal Solution Found', 'Termination condition': 'optimal', 'Id': 0, 'Error rc': 0, 'Time': 0.0184783935546875}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [57]:
# Estimated parameters
m.k1(), m.c0(), m.c1(), m.c2(), m.obj()

(0.4348903894590993,
 0.6760460960475564,
 0.5651096105409008,
 0.3193488719256722,
 0.032439570398357945)

In [54]:
solve_result, inv_red_hes = inv_reduced_hessian_barrier(m, 
                    independent_variables= [m.k1],
                    tee=True)

Ipopt 3.14.5: bound_relax_factor=0
honor_original_bounds=no


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.5, running with linear solver ma27.

Number of nonzeros in equality constraint Jacobian...:        7
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        3

Total number of variables............................:        5
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        5
                     variables with only upper bounds:        0
Total number of equality constraints...